<a href="https://colab.research.google.com/github/ahmabboud/Pizza-Coocking-Level-Recognition/blob/main/Pizza_Automation_Logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pizza Automation Logic

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from fastai import *
from fastai.callbacks.hooks import *
from fastai.vision import *
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import scipy.ndimage

import fastai
import cv2
import os

import json
from datetime import datetime, timedelta
import time

import warnings
warnings.filterwarnings("ignore")

# Main code

In [ ]:
def get_video_text(N, PATH_TO_FILES):
    # ordering of labels
    ORDERING = {
        'frozen': 0,
        'light': 1,
        'medium': 2,
        'dark': 3,
    }

    # get all frames & metadata from path and save in a dict
    result_dict = {}
    for file_name in os.listdir(PATH_TO_FILES):
        if file_name.endswith('.json'):
            full_path_to_file = os.path.join(PATH_TO_FILES, file_name)
            with open(full_path_to_file) as f:
                meta_data_dict = json.load(f)
                cooking_date = int(meta_data_dict['Metadata']['epochtimestamp'])
                datetime_object =  datetime(*time.gmtime(cooking_date/1000)[:6])
                result_dict[file_name[:-5] + '.jpg'] = datetime_object

    # sort result dict to get all frames in chronological order
    sorted_result_dict = sorted(result_dict.items(), key=lambda x: x[1])

    # get time from first frame (as the starting time value)
    first_frame_time = sorted_result_dict[0][1]
    
    # get max cooking time of pizza (can be changed to average cooking time to frozen pizza, etc)
    MAX_TIME = (sorted_result_dict[-1][1] - first_frame_time).seconds

    # percentage of time spent in each state (hardcoded values)
    LIGHT_PERCENTAGE = 0.11
    MEDIUM_PERCENTAGE = 0.44
    DARK_PERCENTAGE = 0.28

    # calculating start times for each class based on percentages
    START_FOR_LIGHT = MAX_TIME * LIGHT_PERCENTAGE
    START_FOR_MEDIUM = MAX_TIME * (LIGHT_PERCENTAGE + MEDIUM_PERCENTAGE)
    START_FOR_DARK = MAX_TIME * (LIGHT_PERCENTAGE + MEDIUM_PERCENTAGE + DARK_PERCENTAGE)

    # see how much cooking time elapsed
    elapsed_time = timedelta(seconds=0)

    # get width, height of current video frame to create output video with same dims 
    first_frame = cv2.imread(os.path.join(PATH_TO_FILES, sorted_result_dict[0][0]))
    h, w, _ = first_frame.shape

    frames_checked = 0

    # create list of tuples; each elem is (frame, category of result, probability)
    # need this map because if we're averaging a result over N frames, we can't just directly
    # use cv2 putText for each frame -- we need to set the result for all N frames to average
    frame_to_result = []

    # create list of frames that stores N frames
    frame_storage = []

    # create list that stores probabilities of pizza being each class for every N frames
    all_probabilities = []

    # start reading frames
    for img_name, timestamp in sorted_result_dict:
        # read in image as cv2 numpy array
        frame = cv2.imread(os.path.join(PATH_TO_FILES, img_name))

        # convert frame from cv2 numpy array to PIL tensor
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        tensor = pil2tensor(img, dtype=np.uint8)
        tensor = tensor.float() / 255.

        # store as fastai image
        fastai_frame = fastai.vision.image.Image(tensor)

        cv2.putText(frame,
                    'Elapsed Time: ' + str((timestamp - first_frame_time)),
                    (5, 125), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        # we first want to check if oven is empty or full -- load empty/full learner
        empty_full_learner = load_learner('/content/drive/My Drive/platform_pizza_models/', 
                                        'platform_oven_empty_or_full.pkl')
        category, _, probs = empty_full_learner.predict(fastai_frame)

        # check if predicted category is empty
        category = str(category).replace('category', '')
        if category.lower() == 'empty_oven':
            # get probability of empty, save result, and continue
            probability = probs[empty_full_learner.data.classes.index(category)]
            frame_to_result.append((frame, 'empty oven', probability))
            continue
        
        else:
            # now check if what's in oven is pizza or not -- load pizza/nonpizza learner
            pizza_nonpizza_learner = load_learner('/content/drive/My Drive/platform_pizza_models/', 
                                                'platform_pizza_nonpizza.pkl')
            category, _, probs = pizza_nonpizza_learner.predict(fastai_frame)

            # check if predicted category is nonpizza
            category = str(category).replace('category', '')
            if category.lower() == 'non-pizza':
                # get probability of nonpizza, save result, and continue
                probability = probs[pizza_nonpizza_learner.data.classes.index(category)]
                frame_to_result.append((frame, 'not a pizza', probability))
                continue
            
            else:
                # now check if pizza is raw or frozen -- load raw/frozen learner
                raw_frozen_learner = load_learner('/content/drive/My Drive/platform_pizza_models/', 
                                                  'platform_pizza_raw_or_frozen.pkl')
                category, _, probs = raw_frozen_learner.predict(fastai_frame)

                # check if predicted category is raw (labeled as scratch in platform project)
                category = str(category).replace('category', '')
                if category.lower() == 'scratch':
                    # get probability of nonpizza, save result, and continue
                    probability = probs[raw_frozen_learner.data.classes.index(category)]
                    frame_to_result.append((frame, 'raw pizza', probability))
                    continue
                
                else:
                    # now we can look to classify the pizza
                    pizza_classification_learner = load_learner('/content/drive/My Drive/platform_pizza_models/', 
                                                                'platform_pizza_classification.pkl')
                    category, _, probs = pizza_classification_learner.predict(fastai_frame)

                    if N > 1:
                        coeff = 4
                        coeff_matrix = torch.ones(4)
                        highest_probability_index = torch.argmax(probs)
                        coeff_matrix[highest_probability_index] = coeff / 2.5 # hardcoded value

                        # smooth predictions in favor of frozen
                        if 0 <= elapsed_time.seconds <= START_FOR_LIGHT:
                            coeff_matrix[0] = coeff

                        # smooth predictions slightly in favor of light
                        elif START_FOR_LIGHT <= elapsed_time.seconds <= START_FOR_MEDIUM:
                            coeff_matrix[1] = coeff
                        
                        # smooth predictions slightly in favor of medium
                        elif START_FOR_MEDIUM <= elapsed_time.seconds <= START_FOR_DARK:
                            coeff_matrix[2] = coeff
                        
                        # smooth predictions slightly in favor of dark
                        else:
                            coeff_matrix[3] = coeff
                        
                        probs = torch.nn.functional.softmax(probs * coeff_matrix)

                    # store probabilities and current frame in their corresponding lists
                    all_probabilities.append(probs.tolist())
                    frame_storage.append(frame)
                    
                    # if we've checked N frames (or 0 frames -- first iteration)
                    if frames_checked % N == 0:
                        # take the mean of all probabilities stored so far
                        avg_probabilities = np.array(all_probabilities).mean(axis=0)
                        
                        # get highest probability result & its corresponding category
                        highest_probability_index = np.argmax(avg_probabilities)
                        predicted_class = pizza_classification_learner.data.classes[highest_probability_index]
                        predicted_probability = avg_probabilities[highest_probability_index]

                        if predicted_class == 'scratch': predicted_class = 'frozen'

                        if N > 1:
                            if len(frame_to_result) > 0:
                                prev_prediction = frame_to_result[-1][1].replace('pizza is ', '')
                                if ORDERING[predicted_class] <= ORDERING[prev_prediction]:
                                    predicted_class = prev_prediction

                        # store result into frame map for all N frames
                        for f in frame_storage:
                            frame_to_result.append((f, 'pizza is ' + predicted_class, predicted_probability))
                        
                        # clear both lists
                        all_probabilities = []
                        frame_storage = []
                        
                    # increment frames checked
                    frames_checked += 1
        
        # set elapsed time
        elapsed_time = timestamp - first_frame_time
    
    return w, h, frame_to_result

## Generating video

In [ ]:
# set paths of video

base = '/content/drive/My Drive/2_oven_validation_dataset/fld/oven-camera-validation/'
path1 = os.path.join(base, 'D828C929CA75/c1e587aa-580e-4485-905c-4ca611b42493')
path2 = os.path.join(base, 'D828C929E2F9/962ffbd5-cb2e-4b0c-8e69-a22262250b24')
path3 = os.path.join(base, 'D828C933D251/ef5e733c-7d7c-42b4-9901-ffac4dcdbfdb')
path4 = os.path.join(base, 'D828C936B06D/0be853f9-a169-461d-b9c0-0e032bc78aa5')
path5 = os.path.join(base, NEW PATH THAT YOU WANT)

In [ ]:
_, _, results = get_video_text(1, path5)
for frame, category, probability in results:
    # show frame
    print(category, probability)

In [ ]:
w, h, N1 = get_video_text(1, path4)
_, _, N2 = get_video_text(3, path4)
fps = 1

out_name = '/content/all_models_result_with_and_without_logic.avi'

# delete video if it already exists
if os.path.exists(out_name): os.remove(out_name)

# create output video
out = cv2.VideoWriter(out_name, 
                      cv2.VideoWriter_fourcc(*'MJPG'),
                      fps, (w, h))

# now, loop through the dict of frames we've stored and create output video
for ((frame, category1, probability1), (_, category2, probability2)) in zip(N1, N2):
    probability1 = str(probability1).replace('tensor(', '').replace(')', '')
    probability2 = str(probability2).replace('tensor(', '').replace(')', '')

    cv2.putText(frame, 
                'Vision model: ' + category1 + ' (prob ' + str(round(float(probability1), 3)) + ')',
                (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
    
    cv2.putText(frame, 
                'Vision model + pure logic: ' + category2 + ' (prob ' + str(round(float(probability2), 3)) + ')',
                (5, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
    
    # put HMM text at (5, 100)
    
    out.write(frame)

# we're done with the output video
out.release()

converted_name = out_name[:-4] + '.mp4'

# delete mp4 file if it already exists
if os.path.exists(converted_name): os.remove(converted_name)

# convert avi to mp4 -- CHANGE VIDEO NAME IF IT WAS CHANGED ABOVE
!ffmpeg -i /content/all_models_result_with_and_without_logic.avi /content/all_models_result_with_and_without_logic.mp4

# remove avi file
os.remove(out_name)

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

# using pandas

In [ ]:
import pandas as pd

In [ ]:
PATH_TO_FILES = '/content/drive/My Drive/2_oven_validation_dataset/fld/' \
'oven-camera-validation/D828C929CA75/c1e587aa-580e-4485-905c-4ca611b42493'

# get all frames & metadata from path and save in a dict
result_dict = {}
for file_name in os.listdir(PATH_TO_FILES):
    if file_name.endswith('.json'):
        full_path_to_file = os.path.join(PATH_TO_FILES, file_name)
        with open(full_path_to_file) as f:
            meta_data_dict = json.load(f)
            cooking_date = int(meta_data_dict['Metadata']['epochtimestamp'])
            datetime_object =  datetime(*time.gmtime(cooking_date/1000)[:6])
            result_dict[datetime_object] = file_name[:-5] + '.jpg'

# sort result dict to get all frames in chronological order
sorted_result_dict = sorted(result_dict.items(), key=lambda x: x[0])
sorted_result_keys = [x[0] for x in sorted_result_dict]
sorted_result_values = [x[1] for x in sorted_result_dict]

In [ ]:
df = pd.DataFrame(OrderedDict({'time': pd.Series(sorted_result_keys), 
                               'frame': pd.Series(sorted_result_values)}))
df

,time,frame
0,2020-08-10 22:40:31,8837b2f2-6f16-4e42-889c-5f89147ba47d.jpg
1,2020-08-10 22:40:41,e80c81d9-8c6b-4118-8ef7-ebad990e5548.jpg
2,2020-08-10 22:40:51,6fed65f6-e96a-45ff-88a7-4102af79f98c.jpg
3,2020-08-10 22:41:01,0d552bec-c6e2-4541-9410-8d7aea7c7187.jpg
4,2020-08-10 22:41:11,ce1ce25e-91f9-491e-b624-873db0e5c97d.jpg
...,...,...
66,2020-08-10 22:58:20,54d4b73b-4539-4f6d-b364-858d9ddfd7a8.jpg
67,2020-08-10 22:58:30,591ed995-b175-4c82-8521-4cc81ac0b0aa.jpg
68,2020-08-10 22:58:40,ba709a3c-120c-4ec4-808a-cf6f92401b1b.jpg
69,2020-08-10 22:58:50,4ce383c8-ebfd-4e61-b0d1-6e2d1b1fe45d.jpg


In [ ]:
df['time'] = pd.to_datetime(df['time'])
df['time'] = df['time'] - df.iloc[0][0]
df

,time,frame
0,00:00:00,8837b2f2-6f16-4e42-889c-5f89147ba47d.jpg
1,00:00:10,e80c81d9-8c6b-4118-8ef7-ebad990e5548.jpg
2,00:00:20,6fed65f6-e96a-45ff-88a7-4102af79f98c.jpg
3,00:00:30,0d552bec-c6e2-4541-9410-8d7aea7c7187.jpg
4,00:00:40,ce1ce25e-91f9-491e-b624-873db0e5c97d.jpg
...,...,...
66,00:17:49,54d4b73b-4539-4f6d-b364-858d9ddfd7a8.jpg
67,00:17:59,591ed995-b175-4c82-8521-4cc81ac0b0aa.jpg
68,00:18:09,ba709a3c-120c-4ec4-808a-cf6f92401b1b.jpg
69,00:18:19,4ce383c8-ebfd-4e61-b0d1-6e2d1b1fe45d.jpg


In [ ]:
all_preds = []
all_probs = []

for timestamp, img_name in sorted_result_dict:
    # read in image as cv2 numpy array
    frame = cv2.imread(os.path.join(PATH_TO_FILES, img_name))

    # convert frame from cv2 numpy array to PIL tensor
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    tensor = pil2tensor(img, dtype=np.uint8)
    tensor = tensor.float() / 255.

    # store as fastai image
    fastai_frame = fastai.vision.image.Image(tensor)

    # we first want to check if oven is empty or full -- load empty/full learner
    empty_full_learner = load_learner('/content/drive/My Drive/platform_pizza_models/', 
                                      'platform_oven_empty_or_full.pkl')
    category, _, probs = empty_full_learner.predict(fastai_frame)

    # check if predicted category is empty
    category = str(category).replace('category', '')
    if category.lower() == 'empty_oven':
        # get probability of empty, save result, and continue
        probability = probs[empty_full_learner.data.classes.index(category)]
        all_preds.append('empty oven')
        all_probs.append(probs)
        continue
    
    else:
        # now check if what's in oven is pizza or not -- load pizza/nonpizza learner
        pizza_nonpizza_learner = load_learner('/content/drive/My Drive/platform_pizza_models/', 
                                              'platform_pizza_nonpizza.pkl')
        category, _, probs = pizza_nonpizza_learner.predict(fastai_frame)

        # check if predicted category is nonpizza
        category = str(category).replace('category', '')
        if category.lower() == 'non-pizza':
            # get probability of nonpizza, save result, and continue
            probability = probs[pizza_nonpizza_learner.data.classes.index(category)]
            all_preds.append('not a pizza')
            all_probs.append(probs)
            continue
        
        else:
            # now check if pizza is scratch or frozen -- load scratch/frozen learner
            raw_frozen_learner = load_learner('/content/drive/My Drive/platform_pizza_models/', 
                                              'platform_pizza_raw_or_frozen.pkl')
            category, _, probs = raw_frozen_learner.predict(fastai_frame)

            # check if predicted category is raw (labeled as scratch in platform project)
            category = str(category).replace('category', '')
            if category.lower() == 'scratch':
                # get probability of scratch pizza, save result, and continue
                probability = probs[raw_frozen_learner.data.classes.index(category)]
                all_preds.append('raw pizza')
                all_probs.append(probs)
                continue
            
            else:
                # now we can look to classify the pizza
                pizza_classification_learner = load_learner('/content/drive/My Drive/platform_pizza_models/', 
                                                            'platform_pizza_classification.pkl')
                category, _, probs = pizza_classification_learner.predict(fastai_frame)

                category = str(category).replace('category', '')
                if category.lower() == 'raw': category = 'frozen'
                probability = probs[pizza_classification_learner.data.classes.index(category)]

                all_preds.append(category)
                all_probs.append(probs)

In [ ]:
df['pred'] = all_preds
df['prob'] = all_probs
#df['raw prob']    = [float(str(x[0]).replace('tensor(', '').replace(')', '')) for x in all_probs]
#df['light prob']  = [float(str(x[1]).replace('tensor(', '').replace(')', '')) for x in all_probs]
#df['medium prob'] = [float(str(x[2]).replace('tensor(', '').replace(')', '')) for x in all_probs]
#df['dark prob']   = [float(str(x[3]).replace('tensor(', '').replace(')', '')) for x in all_probs]
df

,time,frame,pred,prob
0,00:00:00,8837b2f2-6f16-4e42-889c-5f89147ba47d.jpg,light,"[tensor(0.0016), tensor(0.5834), tensor(0.0483..."
1,00:00:10,e80c81d9-8c6b-4118-8ef7-ebad990e5548.jpg,light,"[tensor(0.0009), tensor(0.8420), tensor(0.0315..."
2,00:00:20,6fed65f6-e96a-45ff-88a7-4102af79f98c.jpg,light,"[tensor(0.0013), tensor(0.8441), tensor(0.0708..."
3,00:00:30,0d552bec-c6e2-4541-9410-8d7aea7c7187.jpg,light,"[tensor(0.0018), tensor(0.7822), tensor(0.0489..."
4,00:00:40,ce1ce25e-91f9-491e-b624-873db0e5c97d.jpg,light,"[tensor(0.0017), tensor(0.8182), tensor(0.0479..."
...,...,...,...,...
66,00:17:49,54d4b73b-4539-4f6d-b364-858d9ddfd7a8.jpg,medium,"[tensor(0.0031), tensor(0.2271), tensor(0.7120..."
67,00:17:59,591ed995-b175-4c82-8521-4cc81ac0b0aa.jpg,medium,"[tensor(0.0038), tensor(0.3539), tensor(0.5910..."
68,00:18:09,ba709a3c-120c-4ec4-808a-cf6f92401b1b.jpg,medium,"[tensor(0.0029), tensor(0.0415), tensor(0.9500..."
69,00:18:19,4ce383c8-ebfd-4e61-b0d1-6e2d1b1fe45d.jpg,medium,"[tensor(0.0010), tensor(0.0131), tensor(0.9823..."


In [ ]:
a = [b for b in df['pred']]

In [ ]:
max_time = df.iloc[-1]['time'].seconds
copy = df.copy()
TIME_WEIGHTAGE = 0.15

c = []

for _, row in copy.iterrows():
    curr_time = row['time'].seconds
    percent_spent = curr_time / (max_time / 4)

    # smooth predictions slightly in favor of raw
    if 0 <= curr_time <= max_time / 4:
        row['prob'][0] = (1 - TIME_WEIGHTAGE) * row['prob'][0] + TIME_WEIGHTAGE * percent_spent

    # smooth predictions slightly in favor of light
    elif max_time / 4 <= curr_time <= 2 * max_time / 4:
        row['prob'][1] = (1 - TIME_WEIGHTAGE) * row['prob'][1] + TIME_WEIGHTAGE * percent_spent
    
    # smooth predictions slightly in favor of medium
    elif 2 * max_time / 4 <= curr_time <= 3 * max_time / 4:
        row['prob'][2] = (1 - TIME_WEIGHTAGE) * row['prob'][2] + TIME_WEIGHTAGE * percent_spent
    
    # smooth predictions slightly in favor of dark
    else:
        row['prob'][3] = (1 - TIME_WEIGHTAGE) * row['prob'][3] + TIME_WEIGHTAGE * percent_spent
    
    c.append(['frozen', 'light', 'medium', 'dark'][torch.argmax(row['prob']).item()])

copy

,time,frame,pred,prob
0,00:00:00,8837b2f2-6f16-4e42-889c-5f89147ba47d.jpg,light,"[tensor(0.0002), tensor(0.5834), tensor(0.0483..."
1,00:00:10,e80c81d9-8c6b-4118-8ef7-ebad990e5548.jpg,light,"[tensor(0.0326), tensor(0.8420), tensor(0.0315..."
2,00:00:20,6fed65f6-e96a-45ff-88a7-4102af79f98c.jpg,light,"[tensor(0.0650), tensor(0.8441), tensor(0.0708..."
3,00:00:30,0d552bec-c6e2-4541-9410-8d7aea7c7187.jpg,light,"[tensor(0.0976), tensor(0.7822), tensor(0.0489..."
4,00:00:40,ce1ce25e-91f9-491e-b624-873db0e5c97d.jpg,light,"[tensor(0.1300), tensor(0.8182), tensor(0.0479..."
...,...,...,...,...
66,00:17:49,54d4b73b-4539-4f6d-b364-858d9ddfd7a8.jpg,medium,"[tensor(0.0031), tensor(0.2271), tensor(0.7120..."
67,00:17:59,591ed995-b175-4c82-8521-4cc81ac0b0aa.jpg,medium,"[tensor(0.0038), tensor(0.3539), tensor(0.5910..."
68,00:18:09,ba709a3c-120c-4ec4-808a-cf6f92401b1b.jpg,medium,"[tensor(0.0029), tensor(0.0415), tensor(0.9500..."
69,00:18:19,4ce383c8-ebfd-4e61-b0d1-6e2d1b1fe45d.jpg,medium,"[tensor(0.0010), tensor(0.0131), tensor(0.9823..."


In [ ]:
for b, d in zip(a, c):
    print(b + '\t' + d)

light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
medium	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	light
light	medium
light	medium
light	medium
light	medium
light	medium
light	medium
light	medium
light	medium
light	medium
light	medium
medium	medium
light	medium
light	medium
light	medium
medium	medium
light	medium
light	medium
light	medium
light	medium
light	medium
light	medium
light	medium
light	medium
light	medium
light	dark
light	dark
light	dark
light	dark
medium	dark
medium	dark
medium	dark
medium	dark
medium	dark
medium	dark
medium	dark
medium	dark


In [ ]:
from datetime import timedelta

a = df.groupby(by=(lambda x: df.iloc[x]['time'] <= timedelta(seconds=120)))
b = df.groupby(by=(lambda x: df.iloc[x]['time'] >= timedelta(seconds=120) and df.iloc[x]['time'] <= timedelta(seconds=360)))
c = df.groupby(by=(lambda x: df.iloc[x]['time'] >= timedelta(seconds=360) and df.iloc[x]['time'] <= timedelta(seconds=780)))
d = df.groupby(by=(lambda x: df.iloc[x]['time'] >= timedelta(seconds=780)))

print(a.mean())
print(b.mean())
print(c.mean())
print(d.mean())

       raw prob  light prob  medium prob  dark prob
False  0.001726    0.606119     0.299453   0.092697
True   0.001062    0.792208     0.058154   0.148600
       raw prob  light prob  medium prob  dark prob
False  0.001779    0.607705     0.284283   0.106232
True   0.000225    0.896025     0.026813   0.076950
       raw prob  light prob  medium prob  dark prob
False  0.001209    0.632994     0.272170   0.093633
True   0.001947    0.646442     0.240597   0.111008
       raw prob  light prob  medium prob  dark prob
False  0.001544    0.709340     0.173884   0.115233
True   0.001850    0.358657     0.586636   0.052850


In [ ]:
b = tuple([a for a in df['probs']])
x = torch.stack(b)
x = x.mean(axis=0)
x

tensor([0.0016, 0.6402, 0.2553, 0.1029])